## Scrape Data from Robert Koch Institut
https://experience.arcgis.com/experience/478220a4c454480e823b17327b2bf1d4/page/page_0/

In [2]:
import urllib
import json
import datetime

import pandas

In [6]:
def fetch_data_from_rki(bundesland:str="Hamburg"):
    """
    Fetch Covid-19-Cases from 
    https://experience.arcgis.com/experience/478220a4c454480e823b17327b2bf1d4/page/page_0/
    
    Args:
        bundesland: written like displayed on the website, a string
    Returns:
        a Dataframe containing all historical data from a bundesland
    """
    
    url_endpoint = "https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_COVID19/FeatureServer/0/query"
    params = {
        'f': 'json', 
        'where': f'Bundesland=\'{bundesland}\'',
        'returnGeometry': 'false',
        'spatialRel': 'esriSpatialRelIntersects',
        'outFields': 'ObjectId,AnzahlFall,Meldedatum',
        'orderByFields': 'Meldedatum asc',
        'resultOffset': 0,
        'resultRecordCount': 2000,
        'cacheHint': "true"    
    }

    url_query = f"{url_endpoint}?{urllib.parse.urlencode(params)}"
    print(url_query)

    with urllib.request.urlopen(url_query) as url:
        data = json.loads(url.read().decode())['features']
    
    data_list = [
        (datetime.datetime.fromtimestamp(x['attributes']['Meldedatum'] / 1e3), x['attributes']['AnzahlFall']) 
        for x in data
    ]

    df = pandas.DataFrame(data_list, columns=['Meldedatum', 'AnzahlFall'])

    return df

In [8]:
df = fetch_data_from_rki("Bayern")
df.head()

https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_COVID19/FeatureServer/0/query?f=json&where=Bundesland%3D%27Bayern%27&returnGeometry=false&spatialRel=esriSpatialRelIntersects&outFields=ObjectId%2CAnzahlFall%2CMeldedatum&orderByFields=Meldedatum+asc&resultOffset=0&resultRecordCount=2000&cacheHint=true


,Meldedatum,AnzahlFall
0,2020-01-28 01:00:00,1
1,2020-01-28 01:00:00,1
2,2020-01-29 01:00:00,1
3,2020-01-29 01:00:00,1
4,2020-01-31 01:00:00,1
